In [10]:

from qwen3_moe_fused.fast_lora import patch_Qwen3MoeFusedSparseMoeBlock_forward
from qwen3_moe_fused.lora import patch_lora_config
from qwen3_moe_fused.quantize.quantizer import patch_bnb_quantizer
from qwen3_moe_fused.modular_qwen3_moe_fused import Qwen3MoeFusedForCausalLM
import os

from transformers import BitsAndBytesConfig

from unsloth import FastLanguageModel,FastModel
os.environ.setdefault("TRITON_PRINT_AUTOTUNING", "1")
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally! 

# === Step 1. 打补丁 ===
patch_bnb_quantizer()
patch_lora_config()
patch_Qwen3MoeFusedSparseMoeBlock_forward()


In [12]:
model_id = "/media/gpt4-pdf-chatbot-langchain/transformers-qwen3-moe-fused/moe_unsloth/checkpoint-4900", # YOUR MODEL YOU USED FOR TRAINING


quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="bfloat16",  # 或 torch.float16
)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length = max_seq_length,
    auto_model=Qwen3MoeFusedForCausalLM,
    quantization_config=quant_config,
    trust_remote_code=True,
) 


RuntimeError: Unsloth: No config file found - are you sure the `model_name` is correct?
If you're using a model on your local device, confirm if the folder location exists.
If you're using a HuggingFace online model, check if it exists.

In [ ]:

FastLanguageModel.for_inference(model) # Enable native 2x faster inference




In [ ]:

messages = [
    
            {
                "role": "system",
                "content": "\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Tools\nYou have access to a wide variety of tools. You are responsible for using\nthe tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools\nto complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: resume_zhangmaofeng\nTool Description: 关于张毛峰的简历信息，包括了langchain-chatchat、InterpretationoDreams、KM 平台、省检修特高压生产指挥管控系统、智能运检移动应用、福建监控系统项目eg:检索时带上详细的问题内容\nTool Args: {\"type\": \"object\", \"properties\": {\"input\": {\"title\": \"Input\", \"type\": \"string\"}}, \"required\": [\"input\"]}\n\n> Tool Name: resume_liulijian\nTool Description: 关于刘立兼的简历信息，包括了•篝火心理、雷鸟365、雷鸟365、网聚宝CRM、AP数据基盘、AP数据基盘等项目eg:检索时带上详细的问题内容\nTool Args: {\"type\": \"object\", \"properties\": {\"input\": {\"title\": \"Input\", \"type\": \"string\"}}, \"required\": [\"input\"]}\n\n> Tool Name: resume_songjinke\nTool Description: 关于宋金珂的简历信息，包括了全球 IPv4 空间内的物联网设备扫描识别和隐私安全分、开源软件生态内的跨项目依赖分析及漏洞影响追溯、已发表论文列表、IoT 设备安全等项目eg:检索时带上详细的问题内容\nTool Args: {\"type\": \"object\", \"properties\": {\"input\": {\"title\": \"Input\", \"type\": \"string\"}}, \"required\": [\"input\"]}\n\n\n## Output Format\nTo answer the question, please use the following format.\n\n```\nThought: I need to use a tool to help me answer the question.\nAction: tool name (one of resume_zhangmaofeng, resume_liulijian, resume_songjinke) if using a tool.\nAction Input: the input to the tool, in a JSON format representing the kwargs (e.g. {\"input\": \"hello world\", \"num_beams\": 5})\n```\n\nPlease ALWAYS start with a Thought.\n\nPlease use a valid JSON format for the Action Input. Do NOT do this {'input': 'hello world', 'num_beams': 5}.\n\nIf this format is used, the user will respond in the following format:\n\n```\nObservation: tool response\n```\n\nYou should keep repeating the above format until you have enough information\nto answer the question without using any more tools. At that point, you MUST respond\nin the one of the following two formats:\n\n```\nThought: I can answer without using any more tools.\nAnswer: [your answer here]\n```\n\n```\nThought: I cannot answer the question with the provided tools.\nAnswer: Sorry, I cannot answer your query.\n```\n\n## Current Conversation\nBelow is the current conversation consisting of interleaving human and assistant messages.\n\n"
            },
            {
                "role": "user",
                "content": "请问候选人的全名是什么？"
            },
            {
                "role": "assistant",
                "content": "Thought: 我需要使用工具来获取候选人的全名。\nAction: resume_zhangmaofeng\nAction Input: {'input': '请问候选人的全名是什么？'}"
            }
        
 
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)